In [1]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "0.20"
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
tf.random.set_seed(42)
np.random.seed(42)

In [3]:
X = pd.read_csv('../data/node_data_t.csv', sep=',', encoding='latin-1')
y = pd.read_csv('../data/target_t.csv', sep=',', encoding='latin-1')
X = X[['time_of_day', 'month_of_year', 'node_number']]
y = y[['target']]['target']

In [4]:
X.head()

,time_of_day,month_of_year,node_number
0,2,10,37
1,3,12,37
2,1,12,37
3,2,1,37
4,1,7,37


In [5]:
y.head()

0    0.008940
1    0.007009
2    0.008749
3    0.009498
4    0.010824
Name: target, dtype: float64

In [6]:
def split_data(X, y):
    sep1, sep2, sep3 = int(len(X)*0.7), int(len(X)*0.9), int(len(X))
    X_train, X_test, X_valid = X[:sep1], X[sep1:sep2], X[sep2:sep3]
    # change probability to be percentage
    y_train, y_test, y_valid = y[:sep1]*100, y[sep1:sep2]*100, y[sep2:sep3]*100 
    return X_train, X_test, X_valid, y_train, y_test, y_valid

In [7]:
split_data(X,y)

(     time_of_day  month_of_year  node_number
 0              2             10           37
 1              3             12           37
 2              1             12           37
 3              2              1           37
 4              1              7           37
 ..           ...            ...          ...
 695            3              4           37
 696            2              5           37
 697            1             11           37
 698            1              3           37
 699            2             12           37
 
 [700 rows x 3 columns],
      time_of_day  month_of_year  node_number
 700            3              6           37
 701            3              3           37
 702            2              6           37
 703            1              8           37
 704            3              8           37
 ..           ...            ...          ...
 895            3              2           37
 896            2              1           37
 897   

In [8]:
def build_model(input_shape=[16], n_hidden=1, n_neurons=3, learning_rate=1e-1):
    model = keras.models.Sequential()
    # input layer
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    # hidden layer
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="sigmoid", kernel_initializer="he_normal"))
    # output layer
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mean_squared_error",
                  optimizer=optimizer)
    return model

In [9]:
def train_model(X, y):
    X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(X, y)
    keras_reg = build_model(input_shape=X_train.shape[1:])
    keras_reg.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

    keras_reg.save("neural_network.h5")
    model = keras.models.load_model("neural_network.h5")
    y_pred = model.predict(X_test)
    df = pd.DataFrame(list(zip(y_train, y_pred)), columns=['Actual', 'Predicted'])
    return df

In [35]:
df = train_model(X, y)
print(df.head())

Epoch 1/100
22/22 [==============================] - 0s 4ms/step - loss: 0.3502 - val_loss: 0.0784
Epoch 2/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0657 - val_loss: 0.0797
Epoch 3/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0654 - val_loss: 0.0781
Epoch 4/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0662 - val_loss: 0.0808
Epoch 5/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0658 - val_loss: 0.0780
Epoch 6/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0651 - val_loss: 0.0815
Epoch 7/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0635 - val_loss: 0.0779
Epoch 8/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0641 - val_loss: 0.0791
Epoch 9/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0638 - val_loss: 0.0795
Epoch 10/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0653 - val_loss: 0.0778
Epoch 11/

22/22 [==============================] - 0s 1ms/step - loss: 0.0464 - val_loss: 0.0486
Epoch 84/100
22/22 [==============================] - 0s 1ms/step - loss: 0.0435 - val_loss: 0.0517
Epoch 85/100
7/7 [==============================] - 0s 667us/step
     Actual    Predicted
0  0.893997  [1.0796999]
1  0.700935   [0.822726]
2  0.874891  [1.1389151]
3  0.949796  [1.1095328]
4  1.082431  [1.0079756]


The prediect values are almost always same. To solve this, we should do:
 - scale the data

In [11]:
X.head()

,time_of_day,month_of_year,node_number
0,2,10,37
1,3,12,37
2,1,12,37
3,2,1,37
4,1,7,37


In [12]:
def generate_binary_feature(old_col, value, new_col):
    X[new_col] = X[old_col].apply(lambda x: 1 if x == value else 0)

In [13]:
time_of_day = ['day', 'evening', 'night']
for i in (1,2,3):
    generate_binary_feature('time_of_day', i, time_of_day[i-1])

In [14]:
X.head()

,time_of_day,month_of_year,node_number,day,evening,night
0,2,10,37,0,1,0
1,3,12,37,0,0,1
2,1,12,37,1,0,0
3,2,1,37,0,1,0
4,1,7,37,1,0,0


In [15]:
import calendar
month = list(calendar.month_name)[1:]

In [16]:
for m in range(0, len(month)):
    generate_binary_feature('month_of_year', m+1, month[m])

In [17]:
X['node_number_float'] = X['node_number']/X['node_number'].max()
X.head()

,time_of_day,month_of_year,node_number,day,evening,night,January,February,March,April,May,June,July,August,September,October,November,December,node_number_float
0,2,10,37,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0
1,3,12,37,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.0
2,1,12,37,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0
3,2,1,37,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
4,1,7,37,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0


In [18]:
new_X = X.iloc[:, 3:]

In [19]:
new_X.head()

,day,evening,night,January,February,March,April,May,June,July,August,September,October,November,December,node_number_float
0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1.0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.0
3,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1.0
4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1.0


In [20]:
# df = train_model(new_X, y)
# print(df.head())

The predicted values are not always same anymore. The features are well scaled.
We need to improve the model accuracy.
- Let's try to explore the number of hidden layers, the number of neurons, and the learning rate

In [21]:
new_X.shape[1:]

(16,)

In [22]:
from scipy.stats import reciprocal
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
reciprocal(3e-4, 3e-2).rvs(1000).tolist()

In [37]:
param_distribs ={
    "n_hidden": [1, 2],
#     "n_neurons": np.arange(20, 25),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist()
}

In [39]:
def fine_turn_model(X, y):
    X_train, X_test, X_valid, y_train, y_test, y_valid = split_data(X, y)
#     model = build_model(input_shape=X_train.shape[1:])

    k_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)
    print(k_model)
    search_cv = GridSearchCV(k_model, param_distribs, cv=3)
    search_cv.fit(X_train, y_train, epochs=100, 
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

    y_pred = search_cv.predict(X_test)
    df = pd.DataFrame(list(zip(y_train, y_pred)), columns=['Actual', 'Predicted'])
    return df

In [ ]:
df = fine_turn_model(new_X, y)
print(df.head())

Epoch 1/100


<ipython-input-39-3c92e93da06f>:5: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  k_model = keras.wrappers.scikit_learn.KerasRegressor(build_model)


15/15 [==============================] - 0s 6ms/step - loss: 2.0587 - val_loss: 1.9635
Epoch 2/100
15/15 [==============================] - 0s 2ms/step - loss: 1.9072 - val_loss: 1.8204
Epoch 3/100
15/15 [==============================] - 0s 2ms/step - loss: 1.7679 - val_loss: 1.6885
Epoch 4/100
15/15 [==============================] - 0s 2ms/step - loss: 1.6394 - val_loss: 1.5674
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 1.5213 - val_loss: 1.4557
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 1.4125 - val_loss: 1.3527
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 1.3121 - val_loss: 1.2576
Epoch 8/100
15/15 [==============================] - 0s 2ms/step - loss: 1.2193 - val_loss: 1.1698
Epoch 9/100
15/15 [==============================] - 0s 2ms/step - loss: 1.1336 - val_loss: 1.0888
Epoch 10/100
15/15 [==============================] - 0s 2ms/step - loss: 1.0546 - val_loss: 1.0143
Epoch 11/100
15/15 [=

15/15 [==============================] - 0s 2ms/step - loss: 0.0764 - val_loss: 0.0895
Epoch 84/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0761 - val_loss: 0.0892
Epoch 85/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0759 - val_loss: 0.0890
Epoch 86/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0756 - val_loss: 0.0887
Epoch 87/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0754 - val_loss: 0.0885
Epoch 88/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0752 - val_loss: 0.0883
Epoch 89/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0750 - val_loss: 0.0881
Epoch 90/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0748 - val_loss: 0.0879
Epoch 91/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0746 - val_loss: 0.0878
Epoch 92/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0744 - val_loss: 0.0876
Epoch 93/100


15/15 [==============================] - 0s 2ms/step - loss: 0.0733 - val_loss: 0.0958
Epoch 65/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0732 - val_loss: 0.0958
Epoch 66/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0732 - val_loss: 0.0957
Epoch 67/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0731 - val_loss: 0.0956
Epoch 68/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0731 - val_loss: 0.0955
Epoch 69/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0730 - val_loss: 0.0955
Epoch 70/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0730 - val_loss: 0.0954
Epoch 71/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0729 - val_loss: 0.0953
Epoch 72/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0729 - val_loss: 0.0953
Epoch 73/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0729 - val_loss: 0.0952
Epoch 74/100


...

15/15 [==============================] - 0s 2ms/step - loss: 0.0785 - val_loss: 0.0935
Epoch 59/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0785 - val_loss: 0.0934
Epoch 60/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0784 - val_loss: 0.0934
Epoch 61/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0784 - val_loss: 0.0933
Epoch 62/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0784 - val_loss: 0.0933
Epoch 63/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0783 - val_loss: 0.0933
Epoch 64/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0783 - val_loss: 0.0932
Epoch 65/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0783 - val_loss: 0.0932
Epoch 66/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0783 - val_loss: 0.0931
Epoch 67/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0782 - val_loss: 0.0931
Epoch 68/100


15/15 [==============================] - 0s 2ms/step - loss: 0.0470 - val_loss: 0.0613
Epoch 40/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0468 - val_loss: 0.0610
Epoch 41/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0465 - val_loss: 0.0607
Epoch 42/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0463 - val_loss: 0.0604
Epoch 43/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0461 - val_loss: 0.0601
Epoch 44/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0459 - val_loss: 0.0598
Epoch 45/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0457 - val_loss: 0.0595
Epoch 46/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0455 - val_loss: 0.0592
Epoch 47/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0452 - val_loss: 0.0589
Epoch 48/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0450 - val_loss: 0.0586
Epoch 49/100


15/15 [==============================] - 0s 2ms/step - loss: 0.0744 - val_loss: 0.0930
Epoch 21/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0740 - val_loss: 0.0926
Epoch 22/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0738 - val_loss: 0.0922
Epoch 23/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0736 - val_loss: 0.0918
Epoch 24/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0733 - val_loss: 0.0915
Epoch 25/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0732 - val_loss: 0.0913
Epoch 26/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0730 - val_loss: 0.0910
Epoch 27/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0728 - val_loss: 0.0908
Epoch 28/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0726 - val_loss: 0.0905
Epoch 29/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0725 - val_loss: 0.0903
Epoch 30/100


15/15 [==============================] - 0s 6ms/step - loss: 0.4177 - val_loss: 0.3216
Epoch 2/100
15/15 [==============================] - 0s 2ms/step - loss: 0.2798 - val_loss: 0.2214
Epoch 3/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1956 - val_loss: 0.1604
Epoch 4/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1439 - val_loss: 0.1227
Epoch 5/100
15/15 [==============================] - 0s 2ms/step - loss: 0.1114 - val_loss: 0.1002
Epoch 6/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0917 - val_loss: 0.0866
Epoch 7/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0796 - val_loss: 0.0783
Epoch 8/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0720 - val_loss: 0.0734
Epoch 9/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0672 - val_loss: 0.0702
Epoch 10/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0641 - val_loss: 0.0682
Epoch 11/100
15/15 [=

15/15 [==============================] - 0s 2ms/step - loss: 0.0414 - val_loss: 0.0456
Epoch 84/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0412 - val_loss: 0.0454
Epoch 85/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0410 - val_loss: 0.0452
Epoch 86/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0409 - val_loss: 0.0449
Epoch 87/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0407 - val_loss: 0.0447
Epoch 88/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0405 - val_loss: 0.0445
Epoch 89/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0403 - val_loss: 0.0443
Epoch 90/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0402 - val_loss: 0.0441
Epoch 91/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0400 - val_loss: 0.0439
Epoch 92/100
15/15 [==============================] - 0s 2ms/step - loss: 0.0398 - val_loss: 0.0437
Epoch 93/100


Fine turning model doesn't improve the accuracy of model. 